In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from speck import SpeckCipher
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model

2023-06-09 14:48:41.147113: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
def generate_dataset(num_samples):
    plaintexts = []
    ciphertexts = []
    keys = []

    for _ in range(num_samples):
        # Generate a random 32-bit plaintext
        plaintext = random.randint(0, (1 << 32) - 1)

        # Generate a random 64-bit key
        key = random.randint(0, (1 << 64) - 1)

        # Encrypt the plaintext using Simon block cipher
        cipher = SpeckCipher(key, key_size=64, block_size=32)
        ciphertext = cipher.encrypt(plaintext)

        # Convert the plaintext, ciphertext, and key to binary strings
        plaintext_bits = np.array(list(format(plaintext, '032b')), dtype=int)
        ciphertext_bits = np.array(list(format(ciphertext, '032b')), dtype=int)
        key_bits = np.array(list(format(key, '064b')), dtype=int)

        plaintexts.append(plaintext_bits)
        ciphertexts.append(ciphertext_bits)
        keys.append(key_bits)

    # Convert the lists to NumPy arrays
    plaintexts = np.array(plaintexts)
    ciphertexts = np.array(ciphertexts)
    keys = np.array(keys)

    return plaintexts, ciphertexts, keys
  
    # Generate dataset with 1000 samples
plaintexts, ciphertexts, keys = generate_dataset(8000)


In [3]:
model=load_model('model.h5')

2023-06-09 14:48:43.097938: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-06-09 14:48:43.098476: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-06-09 14:48:43.159959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 14:48:43.160130: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce MX330 computeCapability: 6.1
coreClock: 1.594GHz coreCount: 3 deviceMemorySize: 1.95GiB deviceMemoryBandwidth: 52.21GiB/s
2023-06-09 14:48:43.160153: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-06-09 14:48:43.161685: I tensorflow/stream_executor/platform/default

In [4]:
predicted_keys = model.predict(np.concatenate((plaintexts, ciphertexts), axis=1))

2023-06-09 14:48:43.676986: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-06-09 14:48:43.677404: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 1497600000 Hz
2023-06-09 14:48:43.738475: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


In [5]:
predicted_keys

array([[9.9999917e-01, 0.0000000e+00, 6.2734384e-34, ..., 9.9996340e-01,
        5.1598533e-21, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 2.3281133e-38, ..., 1.0000000e+00,
        0.0000000e+00, 9.9999976e-01],
       [5.5701712e-31, 1.0000000e+00, 1.0000000e+00, ..., 6.1851041e-37,
        1.0000000e+00, 0.0000000e+00],
       ...,
       [1.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.0335747e-31],
       [1.3295968e-19, 1.0926048e-25, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00]], dtype=float32)

In [6]:
loss = model.evaluate(predicted_keys ,keys)
print("Test Loss:", loss)

250/250 [==============================] - 0s 793us/step - loss: 0.4973
Test Loss: 0.49730855226516724


In [8]:
bap =np.mean(np.logical_not(np.logical_xor(keys, np.round(predicted_keys))), axis=0) 
print("Bit Accuracy Probability (BAP):", bap)

Bit Accuracy Probability (BAP): [0.491125 0.50225  0.497375 0.508375 0.505    0.498125 0.50125  0.49525
 0.508    0.499125 0.505375 0.507    0.488625 0.494125 0.507875 0.4965
 0.496375 0.499875 0.51     0.5      0.503125 0.49975  0.49675  0.4985
 0.506    0.503875 0.495    0.49575  0.50925  0.495125 0.497875 0.4985
 0.4955   0.4905   0.49925  0.509375 0.5015   0.501375 0.498    0.508125
 0.51025  0.493875 0.49175  0.507125 0.50175  0.50525  0.506125 0.500625
 0.489375 0.508625 0.4945   0.497    0.502375 0.511625 0.4905   0.500375
 0.506    0.507    0.498    0.496    0.502    0.504625 0.49375  0.499625]
